In [81]:
from langchain_community.document_loaders import CSVLoader
loader = CSVLoader(file_path="Y:/Hackathon/Chennai/Siruseri/team20/Srinidhi/AI Hack/Data/kb_docs.csv", source_column='text')
documents = loader.load()

In [82]:
import numpy as np 
from sentence_transformers import SentenceTransformer
from chromadb.config import Settings 
import uuid 
from typing import List,Dict, Any,Tuple 
from sklearn.metrics.pairwise import cosine_similarity 
import os

### Embeddings

In [83]:
class EmbeddingManager:
    def __init__(self,model_path: str = 'Y:/Hackathon/Chennai/Siruseri/team20/Srinidhi/AI Hack/models/all-MiniLM-L6-v2.pt'):
        self.model_path = model_path
        self.model = None
        self._load_model() 
    def _load_model(self):
        try:
            print(f"Loading model: {self.model_path}")
            self.model = SentenceTransformer(self.model_path)
            print("Model loaded successfully.")
            print(f"Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model: {e}")
            raise e
    def generate_embeddings(self,texts: List[str]) -> np.ndarray:
        if not self.model:
            raise ValueError("Model is not loaded.")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        return embeddings
    def get_embedding_dimension(self) -> int:
        if not self.model:
            raise ValueError("Model is not loaded.")
        return self.model.get_sentence_embedding_dimension()
embedding_manager = EmbeddingManager()
embedding_manager


Loading model: Y:/Hackathon/Chennai/Siruseri/team20/Srinidhi/AI Hack/models/all-MiniLM-L6-v2.pt
Model loaded successfully.
Embedding dimension: 384


In [84]:
import chromadb
import os
import uuid
from typing import List, Any
import numpy as np

### ChromaDb

In [86]:
class VectorStoreDB:
    def __init__(self,collection_name: str = "insurance_knowledge_base",persist_directory: str = "Y:/Hackathon/Chennai/Siruseri/team20/Srinidhi/AI Hack/Data/vector_store"):
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()
    def _initialize_store(self):
        try:
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            self.collection = self.client.get_or_create_collection(name=self.collection_name)
            print(f"ChromaDB collection '{self.collection_name}' initialized at '{self.persist_directory}'.")
        except Exception as e:
            print(f"Error initializing ChromaDB: {e}")
            raise e
    
    def add_documents(self,documents: List[Any],embeddings: np.ndarray):
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents and embeddings must match.")
        print(f"Adding {len(documents)} documents to the vector store...")

        ids=[]
        documents_texts=[]
        embeddings_list=[]
        metadatas=[]
        for i,doc in enumerate(zip(documents, embeddings)):
            doc_obj,embedding=doc
            unique_id = str(uuid.uuid4())
            ids.append(unique_id)
            documents_texts.append(doc_obj.page_content)
            embeddings_list.append(embedding.tolist())
            metadatas.append(doc_obj.metadata)
        try:
            self.collection.add(
                ids=ids,
                documents=documents_texts,
                embeddings=embeddings_list,
                metadatas=metadatas
            )
            print("Documents added successfully.")
        except Exception as e:
            print(f"Error adding documents: {e}")
            raise e
        
vectorestore = VectorStoreDB()
vectorestore._initialize_store()
vectorestore

ChromaDB collection 'insurance_knowledge_base' initialized at 'Y:/Hackathon/Chennai/Siruseri/team20/Srinidhi/AI Hack/Data/vector_store'.
ChromaDB collection 'insurance_knowledge_base' initialized at 'Y:/Hackathon/Chennai/Siruseri/team20/Srinidhi/AI Hack/Data/vector_store'.


In [50]:
texts = [doc.page_content for doc in documents]
embeddings = embedding_manager.generate_embeddings(texts)
vectorestore.add_documents(documents, embeddings)  

Batches: 100%|██████████| 1/1 [00:00<00:00, 11.18it/s]

Adding 8 documents to the vector store...
Documents added successfully.


Documents are stored in vectorDB and ready for RAG retrievel

### RAG Retriever

In [51]:
from typing import List, Dict, Any

class RAGRetriever:
    """
    Retrieval-Augmented Generation retriever for insurance KB.
    Retrieves top-k relevant documents from vector store given a query.
    """
    def __init__(self, vector_store: VectorStoreDB, embedding_manager: EmbeddingManager):
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager
        
    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query.

        Args:
            query (str): Broker/customer question.
            top_k (int): Number of top documents to retrieve.
            score_threshold (float): Minimum similarity score to include document.

        Returns:
            List[Dict[str, Any]]: Retrieved documents with metadata and similarity.
        """
        # 1. Generate embedding for the query
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        try:
            # 2. Query vector store
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )

            retrieved_docs = []

            # Check if any documents were returned
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    similarity_score = 1 - distance  # Chroma distance -> similarity
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            "id": doc_id,
                            "document": document,
                            "metadata": metadata if metadata else {},  # default empty dict
                            "similarity_score": similarity_score,
                            "distance": distance,
                            "rank": i + 1
                        })

            if not retrieved_docs:
                print("No documents found above the score threshold.")

            return retrieved_docs

        except Exception as e:
            print(f"Error during retrieval: {e}")
            raise e

# Instantiate RAG retriever for your CoPilot
rag_retriever = RAGRetriever(vector_store=vectorestore, embedding_manager=embedding_manager)


In [52]:
query = "What insurance products are suitable for a high-risk delivery driver?"
top_docs = rag_retriever.retrieve(query, top_k=3, score_threshold=0.0)

for doc in top_docs:
    print(f"Rank {doc['rank']}, Score: {doc['similarity_score']:.2f}")
    print(doc['document'])
    print("-----")


Batches: 100%|██████████| 1/1 [00:00<00:00, 35.30it/s]

No documents found above the score threshold.


In [53]:
# Count documents in Chroma collection
print("Number of documents in vector store:", vectorestore.collection.count())

Number of documents in vector store: 28


In [54]:
print("First 5 KB documents:")
for doc in documents[:5]:
    print(doc.page_content)


First 5 KB documents:
doc_id: LK001
title: Life Policy Basics
text: Life insurance provides financial protection to your family in case of death of the policyholder.
tags: faq,life
language: en
doc_id: LK002
title: Premium Payment Rules
text: Premiums can be paid monthly
tags: quarterly
language: half-yearly
None: or annually depending on the plan.,faq,life,hi
doc_id: LK003
title: Life Policy Riders
text: Riders like accidental death cover
tags: critical illness
language: and disability cover can be added to enhance protection.
None: coverage,life,en
doc_id: LK004
title: Claim Filing Process
text: Claims must be submitted with death certificate
tags: policy documents
language: and ID proof of nominee.
None: faq,life,ta
doc_id: LK005
title: Maturity Benefits
text: Endowment life policies provide a lump sum at maturity if the policyholder survives the term.
tags: faq,life
language: en


In [55]:
sample_text = documents[0].page_content
embedding = embedding_manager.generate_embeddings([sample_text])
print("Embedding shape:", embedding.shape)


Batches: 100%|██████████| 1/1 [00:00<00:00, 32.87it/s]

Embedding shape: (1, 384)


In [56]:
from sentence_transformers import SentenceTransformer
query = "What insurance products are suitable for a high-risk delivery driver?"

# Generate embedding
query_embedding = embedding_manager.generate_embeddings([query])[0]

# Query Chroma
results = vectorestore.collection.query(
    query_embeddings=[query_embedding.tolist()],
    n_results=3
)

print("Raw query results:", results)


Batches: 100%|██████████| 1/1 [00:00<00:00, 30.92it/s]

Raw query results: {'ids': [['35a9d66e-d75a-4b26-9eee-afe1d488cfa9', '64022916-905f-428f-84ab-83714c42f999', '77229c58-5802-43f7-9023-58a81eb13cc9']], 'embeddings': None, 'documents': [['doc_id: LK003\ntitle: Life Policy Riders\ntext: Riders like accidental death cover\ntags: critical illness\nlanguage: and disability cover can be added to enhance protection.\nNone: coverage,life,en', 'doc_id: KB018\ntitle: Auto Policy Info 18\ntext: Morning hard process happy. Such model lead him. Carry big at gas yard live this.\ntags: coverage,health\nlanguage: en', 'doc_id: KB017\ntitle: Health Policy Info 17\ntext: Mention picture safe or late enter respond. We somebody first system. Sell once their even exactly above. Huge interesting manager job science white series.\ntags: auto,commercial\nlanguage: ta']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'row': 2, 'source': 'Riders like accidental death cover'}, {'source': 'Morning hard process ha

In [57]:
results['documents'][0]

['doc_id: LK003\ntitle: Life Policy Riders\ntext: Riders like accidental death cover\ntags: critical illness\nlanguage: and disability cover can be added to enhance protection.\nNone: coverage,life,en',
 'doc_id: KB018\ntitle: Auto Policy Info 18\ntext: Morning hard process happy. Such model lead him. Carry big at gas yard live this.\ntags: coverage,health\nlanguage: en',
 'doc_id: KB017\ntitle: Health Policy Info 17\ntext: Mention picture safe or late enter respond. We somebody first system. Sell once their even exactly above. Huge interesting manager job science white series.\ntags: auto,commercial\nlanguage: ta']

In [67]:
import pandas as pd
customer_df = pd.read_csv("Y:/Hackathon/Chennai/Siruseri/team20/Srinidhi/AI Hack/Data/customer_crm.csv")
risk_df = pd.read_csv("Y:/Hackathon/Chennai/Siruseri/team20/Srinidhi/AI Hack/Data/risk_profile.csv")
product_df = pd.read_csv("Y:/Hackathon/Chennai/Siruseri/team20/Srinidhi/AI Hack/Data/product_catalog.csv")



In [87]:
customer_profile = customer_df[customer_df['customer_id'] == customer_id].to_dict(orient='records')[0]
risk_profile = risk_df[risk_df['customer_id'] == customer_id].to_dict(orient='records')[0]

In [88]:
customer_profile

{'customer_id': 'LC001',
 'name': 'Aarav Sharma',
 'age': 34,
 'gender': 'M',
 'contact': '+919876543210',
 'language_pref': 'hi',
 'occupation': 'Software Engineer',
 'income_band': '8L-15L',
 'existing_policies': 'Life',
 'claims_history': 1,
 'last_contact_date': '2025-09-20'}

In [89]:
risk_profile

{'customer_id': 'LC001',
 'risk_segment': 'Low',
 'score': 25,
 'drivers': 'Stable income, healthy lifestyle',
 'last_updated': '2025-09-25'}

In [ ]:
customer_id = "LC001"

# Get customer profile
customer_rows = customer_df[customer_df['customer_id'] == customer_id]
customer_profile = customer_rows.to_dict(orient="records")[0]
else:
    customer_profile = None

# Get risk profile
risk_rows = risk_df[risk_df['customer_id'] == customer_id]
if not risk_rows.empty:
    risk_profile = risk_rows.to_dict(orient="records")[0]
else:
    risk_profile = None

print(customer_profile)
print(risk_profile)


{'customer_id': 'LC001', 'name': 'Aarav Sharma', 'age': 34, 'gender': 'M', 'contact': '+919876543210', 'language_pref': 'hi', 'occupation': 'Software Engineer', 'income_band': '8L-15L', 'existing_policies': 'Life', 'claims_history': 1, 'last_contact_date': '2025-09-20'}
{'customer_id': 'LC001', 'risk_segment': 'Low', 'score': 25, 'drivers': 'Stable income, healthy lifestyle', 'last_updated': '2025-09-25'}


In [69]:
print(risk_df['customer_id'].unique())


['LC001' 'LC002' 'LC003' 'LC004' 'LC005' 'LC006' 'LC007' 'LC008' 'LC009'
 'LC010']


In [70]:
customer_profile

{'customer_id': 'LC001',
 'name': 'Aarav Sharma',
 'age': 34,
 'gender': 'M',
 'contact': '+919876543210',
 'language_pref': 'hi',
 'occupation': 'Software Engineer',
 'income_band': '8L-15L',
 'existing_policies': 'Life',
 'claims_history': 1,
 'last_contact_date': '2025-09-20'}

In [71]:
risk_profile

{'customer_id': 'LC001',
 'risk_segment': 'Low',
 'score': 25,
 'drivers': 'Stable income, healthy lifestyle',
 'last_updated': '2025-09-25'}

In [72]:
print(product_df.columns)

Index(['product_id', 'name', 'category', 'key_features', 'eligibility_rules',
       'price_band', 'exclusions', 'recommended_for_segments'],
      dtype='object')


In [73]:
risk_segment = risk_profile['risk_segment']
candidate_products = product_df[
    product_df['recommended_for_segments'].str.contains(risk_segment, case=False, na=False)
].to_dict(orient='records')
print(candidate_products[:2])


[]


In [74]:
prompt = f"""
You are an AI insurance assistant helping brokers recommend suitable products.

Customer Profile:
Name: {customer_profile['name']}
Age: {customer_profile['age']}
Gender: {customer_profile['gender']}
Occupation: {customer_profile['occupation']}
Income: {customer_profile['income_band']}
Existing Policies: {customer_profile['existing_policies']}
Claims History: {customer_profile['claims_history']}

Risk Profile:
Risk Segment: {risk_profile['risk_segment']}
Score: {risk_profile['score']}
Drivers: {risk_profile['drivers']}

Candidate Products:
"""
for i, prod in enumerate(candidate_products):
    prompt += f"{i+1}. {prod['name']} ({prod['category']}) - {prod['key_features']}\n"

prompt += "\nRelevant KB Documents:\n"
for i, doc in enumerate(top_docs):
    prompt += f"{i+1}. {doc['document']}\n"

prompt += f"\nQuestion: {query}\nAnswer professionally, recommending suitable products based on the customer profile, risk profile, and knowledge base."


In [75]:
print("customer_df column names: ", customer_df.columns)
print("risk_df column names: ", risk_df.columns)
print("product_df column names: ", product_df.columns)

customer_df column names:  Index(['customer_id', 'name', 'age', 'gender', 'contact', 'language_pref',
       'occupation', 'income_band', 'existing_policies', 'claims_history',
       'last_contact_date'],
      dtype='object')
risk_df column names:  Index(['customer_id', 'risk_segment', 'score', 'drivers', 'last_updated'], dtype='object')
product_df column names:  Index(['product_id', 'name', 'category', 'key_features', 'eligibility_rules',
       'price_band', 'exclusions', 'recommended_for_segments'],
      dtype='object')


In [76]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
import httpx
load_dotenv()
client = httpx.Client(verify=False) 

In [77]:
llm = ChatOpenAI(
    base_url="https://genailab.tcs.in/",
    model="azure/genailab-maas-gpt-4o",
    api_key="sk-9Iuu7yap97VGk9mYCVa4pw",
    http_client=client
)

In [78]:
def rag_insurance(query, customer_id, retriever, llm, customer_df, risk_df, product_df, top_k=3):
    """
    End-to-end RAG function for insurance recommendation.
    
    Args:
        query (str): The customer's or broker's question.
        customer_id (str): ID of the customer.
        retriever (RAGRetriever): Your RAG retriever instance.
        llm (ChatOpenAI): The LLM instance.
        customer_df (pd.DataFrame): Customer CRM data.
        risk_df (pd.DataFrame): Customer risk profile data.
        product_df (pd.DataFrame): Product catalog data.
        top_k (int): Number of KB documents to retrieve.
        
    Returns:
        str: LLM-generated personalized insurance recommendation.
    """
    
    # 1. Retrieve top KB documents
    top_docs = retriever.retrieve(query, top_k=top_k)

    # 2. Select customer and risk profile
    customer_profile = customer_df[customer_df['customer_id'] == customer_id].to_dict(orient='records')[0]
    risk_profile = risk_df[risk_df['customer_id'] == customer_id].to_dict(orient='records')[0]

    # 3. Filter candidate products based on risk segment
    risk_segment = risk_profile['risk_segment']
    candidate_products = product_df[
        product_df['recommended_for_segments'].str.contains(risk_segment, case=False, na=False)
    ].to_dict(orient='records')

    # 4. Construct the prompt
    prompt = f"""
You are an AI insurance assistant helping brokers recommend suitable products.

Customer Profile:
Name: {customer_profile['name']}
Age: {customer_profile['age']}
Gender: {customer_profile['gender']}
Occupation: {customer_profile['occupation']}
Income: {customer_profile['income_band']}
Existing Policies: {customer_profile['existing_policies']}
Claims History: {customer_profile['claims_history']}

Risk Profile:
Risk Segment: {risk_profile['risk_segment']}
Score: {risk_profile['score']}
Drivers: {risk_profile['drivers']}

Candidate Products:
"""
    for i, prod in enumerate(candidate_products):
        prompt += f"{i+1}. {prod['name']} ({prod['category']}) - {prod['key_features']}\n"

    prompt += "\nRelevant Knowledge Base Documents:\n"
    for i, doc in enumerate(top_docs):
        prompt += f"{i+1}. {doc['document']}\n"

    prompt += f"\nQuestion: {query}\nAnswer professionally, recommending suitable products based on the customer profile, risk profile, and knowledge base."

    # 5. Call the LLM
    response = llm([HumanMessage(content=prompt)])

    # 6. Return the generated answer
    return response.content


In [80]:
query = "What insurance products are suitable for a high-risk delivery driver?"
customer_id = "LC001"

answer = rag_insurance(
    query=query,
    customer_id=customer_id,
    retriever=rag_retriever,
    llm=llm,
    customer_df=customer_df,
    risk_df=risk_df,
    product_df=product_df,
    top_k=3
)

print("=== Personalized Recommendation ===")
print(answer)


Batches: 100%|██████████| 1/1 [00:00<00:00, 25.78it/s]


No documents found above the score threshold.
=== Personalized Recommendation ===
To ensure Aarav Sharma, a low-risk individual, receives tailored insurance recommendations aligned with his profile, we must consider his occupation, income, and existing life insurance coverage. However, your query about a high-risk delivery driver suggests you may wish to consider recommendations in that context. Allow me to clarify and address both scenarios effectively.

### Recommendations for Aarav Sharma:
Given Aarav’s **low-risk profile** and stable income as a software engineer, the following insurance products are ideal:

1. **Health Insurance with Comprehensive Coverage:**
   Aarav should consider a robust health policy that includes hospitalization, critical illnesses, and outpatient care. A plan offering family floater benefits may also be valuable if he plans to extend the coverage to dependents.

2. **Critical Illness Insurance:**
   Despite a low-risk score, critical illnesses like heart d